In [1]:
import gc
import pandas as pd
import polars as pl
import numpy as np
import datetime as dt

import matplotlib.pyplot as plt
import matplotlib.cm as cm
import seaborn as sns

import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
import plotly.offline

import warnings
warnings.filterwarnings('ignore')

In [17]:
train = pl.read_parquet('../data/train.parquet')
test = pl.read_parquet('../data/test.parquet')

In [18]:
train = train.with_columns(
    ("2023" + "-" + (pl.col("Pmonth").cast(str)) + "-" + (pl.col("Pday").cast(str)) + " " + (pl.col("Phour").cast(str)) + ":" + (pl.col("Pmin").cast(str)))\
    .str.strptime(pl.Datetime, "%Y-%m-%d %H:%M").alias("Pdatetime"),
    ("2023" + "-" + (pl.col("Dmonth").cast(str)) + "-" + (pl.col("Dday").cast(str))+ " " + (pl.col("Dhour").cast(str)) + ":" + (pl.col("Dmin").cast(str)))\
    .str.strptime(pl.Datetime, "%Y-%m-%d %H:%M").alias("Ddatetime")
    )
print(len(train.filter((pl.col("Pdatetime") > pl.col("Ddatetime")))))
train.filter((pl.col("Pdatetime") > pl.col("Ddatetime"))).head()


121


Duration,Distance,PLong,PLatd,DLong,DLatd,Haversine,Pmonth,Pday,Phour,Pmin,PDweek,Dmonth,Dday,Dhour,Dmin,DDweek,Temp,Precip,Wind,Humid,Solar,Snow,GroundTemp,Dust,Pdatetime,Ddatetime
i64,i64,f64,f64,f64,f64,f64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,f64,f64,f64,f64,f64,f64,f64,f64,datetime[μs],datetime[μs]
38,3190,37.550079,127.146912,37.538181,127.131752,1.88063,12,31,23,41,0,1,1,0,20,1,-5.2,0.0,1.6,47.0,0.0,0.0,-5.1,53.0,2023-12-31 23:41:00,2023-01-01 00:20:00
35,130,37.636234,126.918999,37.636234,126.918999,0.0,4,22,9,0,6,4,22,0,35,6,16.9,0.0,0.9,45.0,0.27,0.0,16.4,41.0,2023-04-22 09:00:00,2023-04-22 00:35:00
81,260,37.56599,126.987793,37.56599,126.987793,0.0,4,22,8,19,6,4,22,0,46,6,16.3,0.0,0.8,40.0,0.25,0.0,15.7,33.0,2023-04-22 08:19:00,2023-04-22 00:46:00
63,6440,37.595219,127.085999,37.595219,127.085999,0.0,12,31,23,5,0,1,1,0,9,1,-5.2,0.0,1.6,47.0,0.0,0.0,-5.1,53.0,2023-12-31 23:05:00,2023-01-01 00:09:00
53,4330,37.678677,127.040764,37.678677,127.040764,0.0,4,22,9,8,6,4,22,1,4,6,16.9,0.0,0.9,45.0,0.27,0.0,16.4,41.0,2023-04-22 09:08:00,2023-04-22 01:04:00


In [19]:
train.filter((pl.col("Pmonth") == 1) & (pl.col("Pday") == 1)).head()

Duration,Distance,PLong,PLatd,DLong,DLatd,Haversine,Pmonth,Pday,Phour,Pmin,PDweek,Dmonth,Dday,Dhour,Dmin,DDweek,Temp,Precip,Wind,Humid,Solar,Snow,GroundTemp,Dust,Pdatetime,Ddatetime
i64,i64,f64,f64,f64,f64,f64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,f64,f64,f64,f64,f64,f64,f64,f64,datetime[μs],datetime[μs]
27,3570,37.504494,126.921951,37.524837,126.934906,2.534251,1,1,21,15,0,1,1,21,44,0,-0.9,0.0,1.2,44.0,0.0,0.0,-1.2,35.0,2023-01-01 21:15:00,2023-01-01 21:44:00
13,2280,37.513126,127.10096,37.505692,127.084038,1.70623,1,1,20,26,0,1,1,20,41,0,-0.3,0.0,1.2,40.0,0.0,0.0,-0.8,33.0,2023-01-01 20:26:00,2023-01-01 20:41:00
61,6860,37.544579,126.950218,37.547691,126.919983,2.687959,1,1,15,20,0,1,1,16,27,0,3.0,0.0,2.9,27.0,0.78,0.0,1.4,50.0,2023-01-01 15:20:00,2023-01-01 16:27:00
52,7970,37.601299,126.935349,37.601299,126.935349,0.0,1,1,15,43,0,1,1,16,37,0,3.0,0.0,2.9,27.0,0.78,0.0,1.4,50.0,2023-01-01 15:43:00,2023-01-01 16:37:00
27,2630,37.478058,126.960495,37.478069,126.982033,1.900576,1,1,16,10,0,1,1,16,38,0,3.0,0.0,2.5,28.0,0.51,0.0,0.0,53.0,2023-01-01 16:10:00,2023-01-01 16:38:00


In [15]:
test = test.with_columns(
    ("2023" + "-" + (pl.col("Pmonth").cast(str)) + "-" + (pl.col("Pday").cast(str)) + " " + (pl.col("Phour").cast(str)) + ":" + (pl.col("Pmin").cast(str)))\
    .str.strptime(pl.Datetime, "%Y-%m-%d %H:%M").alias("Pdatetime"),
    pl.when((pl.col("Pmonth") == 12) & (pl.col("Pday") == 31) & (pl.col("Dmonth") == 1) & (pl.col("Dday") == 1)) \
    .then(("2024" + "-" + (pl.col("Dmonth").cast(str)) + "-" + (pl.col("Dday").cast(str))+ " " + (pl.col("Dhour").cast(str)) + ":" + (pl.col("Dmin").cast(str)))\
    .str.strptime(pl.Datetime, "%Y-%m-%d %H:%M")) \
    .otherwise(("2023" + "-" + (pl.col("Dmonth").cast(str)) + "-" + (pl.col("Dday").cast(str))+ " " + (pl.col("Dhour").cast(str)) + ":" + (pl.col("Dmin").cast(str)))\
    .str.strptime(pl.Datetime, "%Y-%m-%d %H:%M")).alias("Ddatetime")
    )
print(len(test.filter((pl.col("Pdatetime") > pl.col("Ddatetime")))))
test.filter((pl.col("Pdatetime") > pl.col("Ddatetime"))).head()

7


Duration,Distance,PLong,PLatd,DLong,DLatd,Haversine,Pmonth,Pday,Phour,Pmin,PDweek,Dmonth,Dday,Dhour,Dmin,DDweek,Temp,Precip,Wind,Humid,Solar,Snow,GroundTemp,Dust,Pdatetime,Ddatetime
i64,i64,f64,f64,f64,f64,f64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,f64,f64,f64,f64,f64,f64,f64,f64,datetime[μs],datetime[μs]
57,17820,37.525116,127.052467,37.571255,127.079803,5.668257,4,22,8,52,6,4,22,0,53,6,16.3,0.0,0.8,40.0,0.25,0.0,15.7,33.0,2023-04-22 08:52:00,2023-04-22 00:53:00
14,2300,37.54073,127.102905,37.54073,127.102905,0.0,4,22,9,28,6,4,22,0,44,6,16.9,0.0,0.9,45.0,0.27,0.0,16.4,41.0,2023-04-22 09:28:00,2023-04-22 00:44:00
15,2730,37.54073,127.102905,37.54073,127.102905,0.0,4,22,9,28,6,4,22,0,44,6,16.9,0.0,0.9,45.0,0.27,0.0,16.4,41.0,2023-04-22 09:28:00,2023-04-22 00:44:00
44,6610,37.57576,127.022835,37.598068,127.039276,2.872589,4,22,8,51,6,4,22,0,37,6,16.3,0.0,0.8,40.0,0.25,0.0,15.7,33.0,2023-04-22 08:51:00,2023-04-22 00:37:00
4,890,37.531013,127.142365,37.534771,127.135239,0.754619,4,22,9,29,6,4,22,0,35,6,16.9,0.0,0.9,45.0,0.27,0.0,16.4,41.0,2023-04-22 09:29:00,2023-04-22 00:35:00


### Note
- 年またぎの場合の自転車の利用時間を考慮して利用時間のカラムを作成
- 4/22のデータにバグがある（利用開始時間が利用終了時間より遅い）
- -> これは削除（テストデータにも含まれているが、一回除去したデータで学習・検証）

In [2]:
train = pl.read_parquet('../data/train.parquet')
test = pl.read_parquet('../data/test.parquet')

In [3]:
train = train.with_columns(
    ("2023" + "-" + (pl.col("Pmonth").cast(str)) + "-" + (pl.col("Pday").cast(str)) + " " + (pl.col("Phour").cast(str)) + ":" + (pl.col("Pmin").cast(str)))\
    .str.strptime(pl.Datetime, "%Y-%m-%d %H:%M").alias("Pdatetime"),
    pl.when((pl.col("Pmonth") == 12) & (pl.col("Pday") == 31) & (pl.col("Dmonth") == 1) & (pl.col("Dday") == 1)) \
    .then(("2024" + "-" + (pl.col("Dmonth").cast(str)) + "-" + (pl.col("Dday").cast(str))+ " " + (pl.col("Dhour").cast(str)) + ":" + (pl.col("Dmin").cast(str)))\
    .str.strptime(pl.Datetime, "%Y-%m-%d %H:%M")) \
    .otherwise(("2023" + "-" + (pl.col("Dmonth").cast(str)) + "-" + (pl.col("Dday").cast(str))+ " " + (pl.col("Dhour").cast(str)) + ":" + (pl.col("Dmin").cast(str)))\
    .str.strptime(pl.Datetime, "%Y-%m-%d %H:%M")).alias("Ddatetime")
    )
train = train.filter(~(pl.col("Pdatetime") > pl.col("Ddatetime")))
print(f'check data: {len(train.filter((pl.col("Pdatetime") > pl.col("Ddatetime"))))}')


check data: 0


In [6]:
train = train.with_columns((train['Ddatetime'] - train['Pdatetime']).alias('duration'))
train = train.with_columns((pl.col("duration") / (60 * 10**6)).cast(pl.Int16).alias('Usage_time'))
#     (pl.col("Ddatetime") - pl.col("Pdatetime")).cast(pl.Int16).alias("Usage Time")
# train.with_columns(nwee.cast(int).alias("Usage"))